# 6.1 데이터 수집
## 6.1.1 크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성

In [ ]:
# 예제 6-1  라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 예제 6-2  webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
# driver = webdriver.Chrome('../chromedriver')
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [ ]:
# 예제 6-3 webdriver로 ‘서울’ 버튼 요소를 찾아 클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

In [ ]:
# 예제 6-4  webdriver로 ‘전체’ 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

In [ ]:
# 예제 6-5  BeautifulSoup으로 HTML 파서 만들기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 예제 6-6  select()를 이용해 원하는 HTML 태그를 모두 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

In [ ]:
# 예제 6-7  태그 문자열 살펴보기
starbucks_soup_list[0]

In [ ]:
# 예제 6-8 스타벅스 매장 정보 샘플 확인
startbucks_store = starbucks_soup_list[0]
name = startbucks_store.select('strong')[0].text.strip()
lat = startbucks_store['data-lat'].strip()
lng = startbucks_store['data-long'].strip()
store_type = startbucks_store.select('i')[0]['class'][0][4:]
address = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)         # 매장명
print(lat)          # 위도
print(lng)          # 경도
print(store_type)   # 매장 타입
print(address)      # 주소
print(tel)          # 전화번호

In [ ]:
# 예제 6-9 서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip();
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append( [ name, lat, lng, store_type, address, tel])

In [ ]:
# 예제 6-10  pandas의 데이터프레임 생성
columns = ['매장명','위도','경도','매장타입', '주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

In [ ]:
# 예제 6-11  데이터프레임의 요약 정보 확인
seoul_starbucks_df.info()

In [ ]:
# 예제 6-12 엑셀로 저장
seoul_starbucks_df.to_excel('./files/seoul_starbucks_list.xlsx', index=False)